# Example of training an INR locally
This notebook provides an example of how to create an INR and train it locally using the tools in this repo.

In [1]:
import pdb
import traceback

import jax
from jax import numpy as jnp
import optax
import wandb

from common_dl_utils.config_creation import Config
import common_jax_utils as cju

wandb.login()

key = jax.random.PRNGKey(12398)
key_gen = cju.key_generator(key)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: simon-martinus-koop (nld). Use `wandb login --relogin` to force relogin
2025-01-30 16:02:12.692175: W external/xla/xla/service/gpu/nvptx_compiler.cc:893] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.6.68. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


We want to train a single INR on `example_data/parrot.png`. We'll use the `CombinedINR` clas from `model_components.inr_modules` together with the `SirenLayer` and `GaussianINRLayer` from `model_components.inr_layers` for the model, and we'll train it using the tools from `inr_utils`.

To do all of this, basically we only need to create a config. We'll use the `common_dl_utils.config_creation.Config` class for this, but this is basically just a dictionary that allows for attribute access-like acces of its elements (so we can do `config.model_type = "CombinedINR"` instead of `config["model_type"] = "CombinedINR"`). You can also just use a dictionary instead.

Then we'll use the tools from `common_jax_utils` to first get a model from this config so we can inspect it, and then just run the experiment specified by the config.

Doing this in a config instead of hard coded might seem like extra work, but consider this:
1. you can serialize this config as a json file or a yaml file to later get the same model and experimental settings back 
   so when you are experimenting with different architectures, if you just store the configs you've used, you can easily recreate previous results
2. when we get to running hyper parameter sweeps, you can easily get these configs (with a pick for the varying hyper parameters) from wandb
   and then run an experiment specified by that config on any machine you want, e.g. on Snellius

In [2]:
config = Config()

# first we specify what the model should look like
config.architecture = './model_components'  # module containing all relevant classes for architectures
# NB if the classes relevant for creating the model are spread over multiple modules, this is no problem
# let config.architecture be the module that contains the "main" model class, and for all other components just specify the module
# or specify the other modules as default modules to the tools in common_jax_utils.run_utils
config.model_type = 'inr_modules.CombinedINR'

config.model_config = Config()
config.model_config.in_size = 3
config.model_config.out_size = 1
config.model_config.terms = [  # CombinedINR uses multiple MLPs and returns the sum of their outputs. These 'terms' are the MLPs
    ('inr_modules.MLPINR.from_config',{
        'hidden_size': 256,
        'num_layers': 5,
        'layer_type': 'inr_layers.SirenLayer',
        'num_splits': 3,
        'activation_kwargs': {'w0':25.},#{'inverse_scale': 5.},
        # 'initialization_scheme':'initialization_schemes.siren_scheme',
        # 'initialization_scheme_kwargs': {'w0': 25.},
        'positional_encoding_layer': ('state_test_objects.py', 'CountingIdentity'),
        # 'post_processor': 'model_components.auxiliary.squeeze_array',
    }),
    # ('inr_modules.MLPINR.from_config',{
    #     'hidden_size': 1024,
    #     'num_layers': 2,
    #     'num_splits': 1,
    #     'layer_type': 'inr_layers.GaussianINRLayer',
    #     'use_complex': False,
    #     'activation_kwargs': {'inverse_scale': 1},
    # })
]

# next, we set up the training loop, including the 'target_function' that we want to mimic
config.trainer_module = './inr_utils/'  # similarly to config.architecture above, here we just specify in what module to look for objects by default
config.trainer_type = 'training.train_with_dataloader_scan'


config.dataloader = 'sdf.SDFDataLoader'

config.dataloader_config = {
    "sdf_name": "Armadillo",
    "batch_size": 1000,
    "keep_aspect_ratio":False

}

config.num_cycles = 40
config.steps_per_cycle = 200


config.loss_evaluator = "losses.SDFLossEvaluator"


config.target_function = 'sdf.SDFDataLoader' #see when config. losseval
config.target_function_config = {
    "sdf_name": "Armadillo",
    "batch_size": 10000,
    "keep_aspect_ratio":False,

}

config.state_update_function = ('state_test_objects.py', 'counter_updater')

config.optimizer = 'adam'  # we'll have to add optax to the additional default modules later
config.optimizer_config = {
    'learning_rate': 5e-5,#1.5e-4
}
config.steps = 20000 #changed from 40000
config.use_wandb = True

# now we want some extra things, like logging, to happen during training
# the inr_utils.training.train_inr function allows for this through callbacks.
# The callbacks we want to use can be found in inr_utils.callbacks
config.after_cycle_callback = 'callbacks.ComposedCallback'
config.after_cycle_callback_config = {
    'callbacks':[
        ('callbacks.print_loss', {'after_every':1}),  # only print the loss every 400th step
        'callbacks.report_loss',  # but log the loss to wandb after every step
        ('callbacks.MetricCollectingCallback', # this thing will help us collect metrics and log images to wandb
             {'metric_collector':'metrics.MetricCollector'}
        ),
        'callbacks.raise_error_on_nan'  # stop training if the loss becomes NaN
    ],
    'show_logs': False
}

config.after_training_callback = ('state_test_objects.py', 'after_training_callback')

config.metric_collector_config = {  # the metrics for MetricCollectingCallback / metrics.MetricCollector
    'metrics':[
        ('metrics.JaccardIndexSDF', {
            'frequency':'every_n_batches',
            'grid_resolution': 100,
            'num_dims': 3,
            'batch_size': 10000
        }),
        ("metrics.SDFReconstructor",
         {
            'frequency':'every_n_batches',
            'grid_resolution': 100,
            'batch_size': 10000,
        }),
        # todo add view rendering here

    ],
    'batch_frequency': 5,  # compute all of these metrics every 400 batches
    'epoch_frequency': 1  # not actually used
}

#config.after_training_callback = None  # don't care for one now, but you could have this e.g. store some nice loss plots if you're not using wandb 
config.optimizer_state = None  # we're starting from scratch

In [3]:
# let's first see if we get the correct model
try:
    inr = cju.run_utils.get_model_from_config_and_key(
        prng_key=next(key_gen),
        config=config,
        model_sub_config_name_base='model',
        add_model_module_to_architecture_default_module=False, # since the model is already in the default module specified by 'architecture',
    )
except Exception as e:
    traceback.print_exc()
    print(e)
    print('\n')
    pdb.post_mortem()

In [4]:
inr

CombinedINR(
  terms=(
    MLPINR(
      layers=(
        CountingIdentity(
          _embedding_matrix=f32[3],
          state_index=StateIndex(
            marker=<object object at 0x794528f17e00>,
            init=i32[]
          )
        ),
        SirenLayer(
          weights=f32[256,3],
          biases=f32[256],
          activation_kwargs={'w0': 12.0}
        ),
        SirenLayer(
          weights=f32[256,256],
          biases=f32[256],
          activation_kwargs={'w0': 12.0}
        ),
        SirenLayer(
          weights=f32[256,256],
          biases=f32[256],
          activation_kwargs={'w0': 12.0}
        ),
        Linear(weights=f32[1,256], biases=f32[1], activation_kwargs={})
      )
    ),
  ),
  post_processor=<function real_part>
)

In [5]:
# check that it works properly
try:
    inr(jnp.zeros(3))
except Exception as e:
    traceback.print_exc()
    print(e)
    print('\n')
    pdb.post_mortem()

In [6]:
inr(jnp.zeros(3))


(Array([0.99375975], dtype=float32), None)

In [7]:
# next we get the experiment from the config using common_jax_utils.run_utils.get_experiment_from_config_and_key
experiment = cju.run_utils.get_experiment_from_config_and_key(
    prng_key=next(key_gen),
    config=config,
    model_kwarg_in_trainer='inr',
    model_sub_config_name_base='model',  # so it looks for "model_config" in config
    trainer_default_module_key='trainer_module',  # so it knows to get the module specified by config.trainer_module
    additional_trainer_default_modules=[optax],  # remember the don't forget to add optax to the default modules? This is that 
    add_model_module_to_architecture_default_module=False,
    initialize=False  # don't run the experiment yet, we want to use wandb
)

In [8]:
experiment

PostponedInitialization(cls=train_with_dataloader_scan, kwargs={'loss_evaluator': PostponedInitialization(cls=SDFLossEvaluator, kwargs={'state_update_function': <function counter_updater at 0x7944b838d900>}, missing_args=[]), 'dataloader': PostponedInitialization(cls=SDFDataLoader, kwargs={'sdf_name': 'Armadillo', 'batch_size': 1000, 'keep_aspect_ratio': False, 'key': Array([1786414058, 1458264990], dtype=uint32)}, missing_args=[]), 'optimizer': PostponedInitialization(cls=adam, kwargs={'learning_rate': 5e-05, 'b1': 0.9, 'b2': 0.999, 'eps': 1e-08, 'eps_root': 0.0, 'mu_dtype': None, 'nesterov': False}, missing_args=[]), 'steps_per_cycle': 200, 'num_cycles': 40, 'use_wandb': True, 'after_cycle_callback': PostponedInitialization(cls=ComposedCallback, kwargs={'callbacks': [functools.partial(<function print_loss at 0x7944b834b370>, after_every=1), <function report_loss at 0x7944b836b400>, PostponedInitialization(cls=MetricCollectingCallback, kwargs={'metric_collector': PostponedInitializati

In [9]:
# and we run the experiment while logging things to wandb
try:
    with wandb.init(
        project='inr_edu_24',
        notes='test',
        tags=['test']
    ) as run:
        results = experiment.initialize()
except Exception as e:
    traceback.print_exc()
    print(e)
    print('\n')
    pdb.post_mortem()


Loss at step 1 is 191.8506622314453.
Loss at step 2 is 103.94175720214844.
Loss at step 3 is 100.90261840820312.
Loss at step 4 is 94.42613220214844.
Loss at step 5 is 87.54032135009766.
Loss at step 6 is 82.86439514160156.
Loss at step 7 is 75.28373718261719.
Loss at step 8 is 68.85980224609375.
Loss at step 9 is 74.43783569335938.
Loss at step 10 is 63.67566680908203.
Loss at step 11 is 60.15531921386719.
Loss at step 12 is 57.67008590698242.
Loss at step 13 is 54.10897445678711.
Loss at step 14 is 52.776161193847656.
Loss at step 15 is 48.430633544921875.
Loss at step 16 is 52.13328552246094.
Loss at step 17 is 52.10590744018555.
Loss at step 18 is 48.91326904296875.
Loss at step 19 is 49.01671600341797.
Loss at step 20 is 49.0701904296875.
Loss at step 21 is 45.68402862548828.
Loss at step 22 is 43.310760498046875.
Loss at step 23 is 46.14805603027344.
Loss at step 24 is 39.01842498779297.
Loss at step 25 is 37.98085403442383.
Loss at step 26 is 45.088748931884766.
Loss at step 27 

batch_within_epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
jaccard_index,▁▅▆█▅▅▅▅
loss,█▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
batch_within_epoch,40
epoch,1
jaccard_index,0.15377
loss,37.61947


In [10]:
inr, losses, optimizer_state, state, loss_evaluator, additional_output = results
inr

CombinedINR(
  terms=(
    MLPINR(
      layers=(
        CountingIdentity(
          _embedding_matrix=f32[3],
          state_index=StateIndex(marker=0, init=_Sentinel())
        ),
        SirenLayer(
          weights=f32[256,3],
          biases=f32[256],
          activation_kwargs={'w0': 12.0}
        ),
        SirenLayer(
          weights=f32[256,256],
          biases=f32[256],
          activation_kwargs={'w0': 12.0}
        ),
        SirenLayer(
          weights=f32[256,256],
          biases=f32[256],
          activation_kwargs={'w0': 12.0}
        ),
        Linear(weights=f32[1,256], biases=f32[1], activation_kwargs={})
      )
    ),
  ),
  post_processor=<function real_part>
)

In [11]:
from state_test_objects import after_training_callback, CountingIdentity
after_training_callback(losses, inr, state)

Checking model and state for CountingIdentity layers
Found a CountingIdentity layer with counter value 0 in final state after training.


In [13]:
from inr_utils.sdf import SDFDataLoader
data_loader = SDFDataLoader(**config.dataloader_config, key=jax.random.PRNGKey(0))

batch = next(iter(data_loader))
batch[0]

Array([[-0.23807062,  0.79148495, -0.02933969],
       [ 0.12659283,  0.8626764 , -0.3414005 ],
       [-0.12326922,  0.8929266 ,  0.26043114],
       ...,
       [ 0.0527916 ,  0.9796984 ,  0.6806431 ],
       [-0.80625963,  0.77745533, -0.24835396],
       [ 0.29667616,  0.21406317,  0.44846964]], dtype=float32)

In [21]:
jax.numpy.linalg.norm(batch[1], axis=-1).min()

Array(0.99999994, dtype=float32)

In [22]:
jax.numpy.linalg.norm(batch[1], axis=-1).max()

Array(1.7320508, dtype=float32)

In [23]:
jax.tree.map(lambda x: x.device, batch)

(CudaDevice(id=0), CudaDevice(id=0), CudaDevice(id=0))

In [24]:
batch[1].shape

(1000, 3)

In [25]:
def get_default_device():
  return jax.config.jax_default_device or jax.local_devices()[0]
get_default_device()

CudaDevice(id=0)